# Export results using different displacement height and roughness length;
- This script is used to export flux variables using different displacement height and roughness length;
- Simulations: CNTL, S1, S2, S3;

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import cftime
import string

In [2]:
archive_path = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/Projects/archive/0project2/'
CNTL = archive_path + '0sp/UK-Kin_def/lnd/hist/UK-Kin_def.clm2.h0.2002-01-01-00000.nc'
S1 = archive_path + '0roughness_sensitivity/UK-Kin_S1/lnd/hist/UK-Kin_S1.clm2.h0.2002-01-01-00000.nc'
S3 = archive_path + '0roughness_sensitivity/UK-Kin_S3/lnd/hist/UK-Kin_S3.clm2.h0.2002-01-01-00000.nc'
datm = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/UrbanPlumber/datm_files/UK-Kin/CLM1PT_data/CTSM_DATM_UK-Kin_2002-2014.nc'
observation = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/UrbanPlumber/Urban-PLUMBER_FullCollection_v1/UK-KingsCollege/timeseries/UK-KingsCollege_clean_observations_v1.nc'
ds_cntl = xr.open_dataset(CNTL)
ds_s1 = xr.open_dataset(S1)
ds_s3 = xr.open_dataset(S3)
ds_datm = xr.open_dataset(datm)
ds_ob = xr.open_dataset(observation)

start_time = '2012-04-04T00:00:00'
end_time = '2013-12-31T21:00:00'
mid_time = ['2013-01-01T00:00:00']

sequence = ['Rn', 'SWup', 'LWup', 'Qh', 'Qle', 'Qtau']
sequence2 = ['$R_{n}$', '$SW_{up}$', '$LW_{up}$', '$Q_{h}$', '$Q_{le}$', '$Q_{tau}$']
var0 = ['LWdown', 'SWdown']
var1 = ['LWdown', 'SWdown','LWup', 'SWup', 'Qh', 'Qle', 'Qtau']
var2 = ['FIRE_U', 'FSR', 'FSH_U', 'EFLX_LH_TOT_U', 'TAUX']
time_zone = 0

In [3]:
ds_cntl['time']=ds_cntl['time'].dt.round('min')
ds_cntl['time']=ds_cntl['time'].dt.ceil('min')
ds_s1['time']=ds_s1['time'].dt.round('min')
ds_s1['time']=ds_s1['time'].dt.ceil('min')
ds_s3['time']=ds_s3['time'].dt.round('min')
ds_s3['time']=ds_s3['time'].dt.ceil('min')

sliced_ds_datm = ds_datm.sel(time=slice(start_time, end_time))[var0]
df_datm = sliced_ds_datm.to_dataframe().reset_index()

sliced_ds_cntl = ds_cntl.sel(time=slice(start_time, end_time))[var2]
df_cntl = sliced_ds_cntl.to_dataframe().reset_index()
df_cntl.drop(columns=['lndgrid'], inplace=True)
df_cntl = df_cntl.rename(columns={'FIRE_U': 'LWup_def', 'FSR': 'SWup_def','FSH_U': 'Qh_def', 'EFLX_LH_TOT_U': 'Qle_def', 'TAUX': 'Qtau_def'})
df_cntl['Qtau_def'] = - df_cntl['Qtau_def']
df_cntl['Rn_def'] = df_datm['LWdown'] + df_datm['SWdown'] - df_cntl['SWup_def'] - df_cntl['LWup_def']

sliced_ds_s1 = ds_s1.sel(time=slice(start_time, end_time))[var2]
df_s1 = sliced_ds_s1.to_dataframe().reset_index()
df_s1.drop(columns=['lndgrid'], inplace=True)
df_s1 = df_s1.rename(columns={'FIRE_U': 'LWup_s1', 'FSR': 'SWup_s1','FSH_U': 'Qh_s1', 'EFLX_LH_TOT_U': 'Qle_s1', 'TAUX': 'Qtau_s1'})
df_s1['Qtau_s1'] = - df_s1['Qtau_s1']
df_s1['Rn_s1'] = df_datm['LWdown'] + df_datm['SWdown'] - df_s1['SWup_s1'] - df_s1['LWup_s1']

sliced_ds_s3 = ds_s3.sel(time=slice(start_time, end_time))[var2]
df_s3 = sliced_ds_s3.to_dataframe().reset_index()
df_s3.drop(columns=['lndgrid'], inplace=True)
df_s3 = df_s3.rename(columns={'FIRE_U': 'LWup_s3', 'FSR': 'SWup_s3','FSH_U': 'Qh_s3', 'EFLX_LH_TOT_U': 'Qle_s3', 'TAUX': 'Qtau_s3'})
df_s3['Qtau_s3'] = - df_s3['Qtau_s3']
df_s3['Rn_s3'] = df_datm['LWdown'] + df_datm['SWdown'] - df_s3['SWup_s3'] - df_s3['LWup_s3']

sliced_ds_ob = ds_ob.sel(time=slice(start_time, end_time))[var1]
df_ob = sliced_ds_ob.to_dataframe().reset_index()
df_ob = df_ob.rename(columns={'LWdown': 'LWdown_obs', 
                                      'SWdown': 'SWdown_obs',
                                      'LWup': 'LWup_obs', 
                                      'SWup': 'SWup_obs',
                                      'Qh': 'Qh_obs', 
                                      'Qle': 'Qle_obs', 
                                      'Qtau': 'Qtau_obs'})
df_ob['Rn_obs'] = df_ob['LWdown_obs'] + df_ob['SWdown_obs']-df_ob['SWup_obs']-df_ob['LWup_obs']

df = pd.merge(df_ob, df_cntl, on='time').merge(df_s1, on='time').merge(df_s3, on='time')
df.loc[df['LWup_obs'].isna(), ['LWup_s1', 'LWup_s2','LWup_def',
                                  'Rn_obs', 'Rn_s1', 'Rn_s3', 'Rn_def']] = np.nan
df.loc[df['SWup_obs'].isna(), ['SWup_s1', 'SWup_s3', 'SWup_def',
                                  'Rn_obs', 'Rn_s1', 'Rn_s3', 'Rn_def']] = np.nan
df.loc[df['Qle_obs'].isna(), ['Qle_s1', 'Qle_s3','Qle_def']] = np.nan
df.loc[df['Qh_obs'].isna(), ['Qh_s1', 'Qh_s3','Qh_def']] = np.nan
df.loc[df['Qtau_obs'].isna(), ['Qtau_s1', 'Qtau_s3','Qtau_def']] = np.nan

df['hour']=(df.time.dt.hour + time_zone) % 24
df['minute']=df.time.dt.minute
df['diurnal']=df['hour'].astype(str).str.zfill(2) + ':' + df['minute'].astype(str).str.zfill(2)
output_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/output_analysis/project2/sp_roughness_sensitivity/kingscollege/'
df.to_csv(output_dir + 'result.csv', index = False)